In [1]:
pip install keras-tuner

     |████████████████████████████████| 98 kB 3.5 MB/s 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pprint import pprint
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import OneHotEncoder
from keras.utils import np_utils
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import gensim
import re
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Embedding
from keras import backend as K
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from tensorflow.keras.callbacks import EarlyStopping


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


#Print result

In [3]:
def printResult(y_real, y_pred):
    acc = accuracy_score(y_real, y_pred)
    print("Accuracy: {:.2f}".format(acc*100),end='\n\n')
    print(classification_report(y_real,y_pred))

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading CBOW  using gensim

In [5]:
model = gensim.models.Word2Vec.load('/content/drive/MyDrive/full_uni_sg_300_twitter.mdl')

# Reading training and testing dataset

In [6]:
#read train dataset
train= pd.read_csv('/content/drive/MyDrive/ML task/Deep/train1.csv',encoding='utf-8')
train=train.dropna()
y_train = train.dialect                   
X_train= train.drop(['dialect','id'],axis = 1 )

In [7]:
#reading testing data
test= pd.read_csv('/content/drive/MyDrive/ML task/Deep/unseen1.csv',encoding='utf-8')
test=test.dropna()
y_test = test.dialect                           
X_test= test.drop(['id','dialect'],axis = 1 )

# Label Encoder 

In [8]:
encoder = LabelEncoder()
y_train= encoder.fit_transform(y_train)
y_train_best=np_utils.to_categorical(y_train)
encoder1 = LabelEncoder()
y_test = encoder1.fit_transform(y_test)
y_test_best=np_utils.to_categorical(y_test)

# Building word embedding

In [9]:
MAX_LEN = 140  
BATCH_SIZE = 5000
MAX_NUM_WORDS = 200000
EMBEDDING_DIM = 300
epoch=100

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(X_train['text'])
train_sequences = tokenizer.texts_to_sequences(X_train['text'])

word_index = tokenizer.word_index
padded_train = pad_sequences(sequences=train_sequences, maxlen=MAX_LEN)
print('Total unique tokens generated: ',len(word_index))
print('Shape of padded train tensor: ', padded_train.shape)
test_sequences = tokenizer.texts_to_sequences(X_test['text'])
padded_test = pad_sequences(sequences=test_sequences, maxlen=MAX_LEN)

print('Shape of padded test tensor: ', padded_test.shape)

Total unique tokens generated:  50714
Shape of padded train tensor:  (12361, 140)
Shape of padded test tensor:  (1375, 140)


In [10]:
num_words = min(MAX_NUM_WORDS, len(word_index))
word_embedding_matrix = np.zeros((num_words + 1, EMBEDDING_DIM))

for word, index in word_index.items():
    if index > MAX_NUM_WORDS:
        continue
    if word not in model.wv:
        embedding_vector = None
    else:
        embedding_vector = model.wv[word]
    if embedding_vector is not None:
        word_embedding_matrix[index] = embedding_vector

In [11]:
output=len(train['dialect'].value_counts())

# Buliding deep learning using keras tuner

In [12]:
def build_model(hp):
    num_units_min  =  50
    num_units_max  =  1000
    num_units_step =  50

    dropout_min  =  .1
    dropout_max  =  0.9
    dropout_step =  0.1
    
    model = keras.Sequential()
 
    model.add(layers.Embedding(num_words + 1, EMBEDDING_DIM, weights=[word_embedding_matrix], input_length=MAX_LEN))

    
    model.add(layers.GRU(units=hp.Int('unit1',  min_value=num_units_min,
                                                 max_value=num_units_max,
                                                 step=num_units_step), return_sequences =False,  activation='relu', kernel_initializer='he_uniform'))
    model.add(layers.Dense(output, activation='softmax'))
    model.compile( optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[ 1e-4])),loss='categorical_crossentropy', metrics=['acc'])
    return model


In [13]:
tuner = RandomSearch(
    build_model,
    objective='val_acc',
    max_trials=1,
    project_name='/content/drive/MyDrive/ML task/Deep/GRU_sg')


In [14]:
early_stopping = EarlyStopping(monitor='val_acc', patience=30, verbose=1)
callback_list = [ early_stopping ]

h=tuner.search(padded_train, y_train_best,
             epochs=epoch,
             batch_size=BATCH_SIZE, 
             callbacks=callback_list, validation_split=0.1)         


Trial 1 Complete [00h 04m 26s]
val_acc: 0.3225545585155487

Best val_acc So Far: 0.3225545585155487
Total elapsed time: 00h 04m 26s
INFO:tensorflow:Oracle triggered exit


In [15]:
model = tuner.get_best_models(num_models=1)[0]

In [16]:
model.save("/content/drive/MyDrive/ML task/Deep/GRU_sg.h5")

In [17]:
tuner.get_best_hyperparameters()[0].values

{'learning_rate': 0.0001, 'unit1': 250}

#Print training result

In [18]:
y_train_pred=model.predict(padded_train)
y_pred= np.argmax(y_train_pred, axis=1)
print("Train Result of GRU")
printResult(y_train,y_pred)

Train Result of GRU
Accuracy: 50.09

              precision    recall  f1-score   support

           0       0.40      0.28      0.33       709
           1       0.40      0.35      0.37       710
           2       0.59      0.28      0.37       436
           3       0.70      0.93      0.80      1556
           4       0.83      0.20      0.32       416
           5       0.39      0.35      0.37       754
           6       0.48      0.67      0.56      1137
           7       0.68      0.48      0.56       746
           8       0.39      0.75      0.51       983
           9       0.79      0.18      0.29       311
          10       0.37      0.15      0.21       517
          11       0.46      0.74      0.57      1181
          12       0.43      0.60      0.50       838
          13       0.49      0.44      0.46       724
          14       0.61      0.14      0.23       390
          15       0.64      0.17      0.27       436
          16       0.33      0.00      0.01 

# Print testing result

In [19]:
y_test_pred=model.predict(padded_test)
y_pred= np.argmax(y_test_pred, axis=1)
print("Testing Result of GRU")
printResult(y_test,y_pred)

Testing Result of GRU
Accuracy: 29.67

              precision    recall  f1-score   support

           0       0.16      0.10      0.12        79
           1       0.12      0.11      0.12        79
           2       0.25      0.08      0.12        48
           3       0.57      0.76      0.65       173
           4       0.00      0.00      0.00        46
           5       0.10      0.10      0.10        84
           6       0.32      0.41      0.36       126
           7       0.41      0.24      0.30        83
           8       0.22      0.50      0.30       110
           9       0.67      0.06      0.11        35
          10       0.14      0.05      0.08        57
          11       0.27      0.50      0.35       131
          12       0.26      0.32      0.29        93
          13       0.29      0.23      0.26        81
          14       0.17      0.02      0.04        43
          15       0.00      0.00      0.00        49
          16       0.00      0.00      0.0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
